In [350]:
import pandas as pd
import numpy as np
from glob import glob
import dill

## Clean location information

Import and clean all of the "Places" csv files from the [GitHub repo](https://github.com/cdcepi/zika), which is the `zika` directory below.

In [312]:
location_files = glob('../zika/*/*Places.csv')
locations = pd.concat([pd.read_csv(x) for x in location_files], axis=0).reset_index(drop=True)

In [313]:
# Data that will be difficult to incorporate into the model
mask = locations.location_type.isin(['country', 'region', 'district']).pipe(np.invert)
locations = locations.loc[mask]

In [314]:
locations = locations.dropna(axis=1, how='all')

In [315]:
location_key = locations[['location', 'location_type']]
location_key[['country', 'province', 'county', 'city']] = location_key.location.str.split(r"""-""", expand=True)

In [316]:
def map_locations(x):
    location_mapper = {'state':'province',
                       'municipality':'city',
                       'department':'province',
                       'Region':'province',
                       'Collectivity':'province',
                       'territory':'province'
                      }
    if x in location_mapper.keys():
        return location_mapper[x]
    else:
        return x
    
location_key['location_type'] = location_key.location_type.apply(lambda x: map_locations(x))

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [317]:
# Fix the US Virgin Islands entries
mask = ( location_key.county.isnull() & 
         (location_key.location_type=='county') &
         (location_key.country=='United_States_Virgin_Islands')
        )

location_key.loc[mask, 'county'] = location_key.loc[mask, 'province']
location_key.loc[mask, 'province'] = 'Virgin Islands'
location_key.loc[mask, 'country'] = 'United States'


mask = ( location_key.province.isnull() & 
         (location_key.location_type=='province'))
location_key.loc[mask, 'province'] = 'Virgin Islands'
location_key.loc[mask, 'country'] = 'United States'

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.

In [318]:
# Fix remaining counties (mainly in Ecuador and Panama)
mask = ( location_key.county.isnull() & 
         (location_key.location_type=='county'))

location_key.loc[mask, 'county'] = location_key.loc[mask, 'province']
location_key.loc[mask, 'province'] = None

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [319]:
# Move cities to correct column
mask = ( location_key.city.isnull() & 
         (location_key.location_type=='city'))

location_key.loc[mask, 'city'] = location_key.loc[mask, 'county']
location_key.loc[mask, 'county'] = None

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [320]:
# More fixes for cities
mask = ( location_key.city.isnull() & 
         (location_key.location_type=='city'))

location_key.loc[mask, 'city'] = location_key.loc[mask, 'province']
location_key.loc[mask, 'province'] = None

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [329]:
# Drop unknown cities
location_key = location_key[location_key.city.isin(['Unknown','Not_Reported']).pipe(np.invert)]

In [325]:
# Fix for Dade County Florida
mask = location_key.location=='United_States-Florida-Miami-Dade_County'
location_key.loc[mask, 'county'] = 'Dade_County'
location_key.loc[mask, 'city'] = 'Miami'

In [340]:
# Fix for Santiago Del Estero Argentina
location_key.loc[location_key.location=='Argentina-Sgo_Del_Estero', 'province'] = 'Santiago Del Estero'
location_key.loc[location_key.location=='Argentina-CABA', 'province'] = 'Ciudad de Buenos Aires'

In [341]:
# Remove county name
location_key['county'] = location_key.county.str.replace('_County','')

location_key = location_key[location_key.county.isin(['Unknown','Not_Reported']).pipe(np.invert)]

In [342]:
# Remove all underscores
for col in ['country', 'province', 'county', 'city']:
    location_key[col] = location_key[col].str.replace('_', ' ')

In [349]:
# For checking the data 50 lines at a time
# i=31
# nsize = 50
# location_key.iloc[i*nsize:(i+1)*nsize]

In [359]:
location_key.to_csv('../csv/00_cleaned_city_names.csv', sep=',', index=False)

In [358]:
with open('../pkl/00_cleaned_city_names.pkl', 'w') as fh:
    dill.dump(location_key, fh)